In [22]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd
import numpy as np

# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
#personalize.list_recipes()
%store -r

In [23]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)
bucket_name='657975005475-cn-north-1-personalizepocvod'
role_name = "PersonalizeRolePOC"
role_arn='arn:aws-cn:iam::657975005475:role/PersonalizeRolePOC'
dataset_group_arn ='arn:aws-cn:personalize:cn-north-1:657975005475:dataset-group/bestv-vod-item-summary'

cn-north-1


In [4]:
interactions_filename = "interactions.csv"
#boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

itemmetadata_filename = "item-meta.csv"
#boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_filename)
itemmetadata_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

In [2]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "bestv-vod-item-summary"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws-cn:personalize:cn-north-1:657975005475:dataset-group/bestv-vod-item-summary",
  "ResponseMetadata": {
    "RequestId": "47fe443c-3046-49ff-befd-f6e088416717",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 04:36:48 GMT",
      "x-amzn-requestid": "47fe443c-3046-49ff-befd-f6e088416717",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [7]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: ACTIVE


In [4]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "content_count",
            "type": "int"
        },
        {
            "name": "play_duration",
            "type": "int"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "package_interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws-cn:personalize:cn-north-1:657975005475:schema/package_interactions",
  "ResponseMetadata": {
    "RequestId": "040d7456-8cd6-4eab-8ddd-3d0f15c0de6f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 04:39:09 GMT",
      "x-amzn-requestid": "040d7456-8cd6-4eab-8ddd-3d0f15c0de6f",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [5]:
#dataset_group_arn='arn:aws-cn:personalize:cn-north-1:657975005475:dataset-group/bestv-vod'
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "package-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws-cn:personalize:cn-north-1:657975005475:dataset/bestv-vod-item-summary/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "f115c03b-772c-46e8-9486-db0e4001e2c0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 04:39:47 GMT",
      "x-amzn-requestid": "f115c03b-772c-46e8-9486-db0e4001e2c0",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [12]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "package-import1",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws-cn:personalize:cn-north-1:657975005475:dataset-import-job/package-import1",
  "ResponseMetadata": {
    "RequestId": "e415cb35-1ab5-4aaf-9a5b-7e6f26077c60",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 04:45:01 GMT",
      "x-amzn-requestid": "e415cb35-1ab5-4aaf-9a5b-7e6f26077c60",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [13]:

%store interactions_dataset_arn
%store dataset_group_arn
%store bucket_name
%store role_arn
%store role_name
%store region
%store interaction_schema_arn

Stored 'interactions_dataset_arn' (str)
Stored 'dataset_group_arn' (str)
Stored 'bucket_name' (str)
Stored 'role_arn' (str)
Stored 'role_name' (str)
Stored 'region' (str)
Stored 'interaction_schema_arn' (str)


In [14]:
itemmetadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "REGION",
            "type": "string"
        },
        {
            "name": "YEAR",
            "type": "int"
        },
        {
            "name": "TYPE",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string"
        },
        {
            "name": "ACTORS",
            "type": "string",
            "categorical": True
        },
        {
            "name": "WRITERS",
            "type": "string",
            "categorical": True
        },
        {
            "name": "TAGS",
            "type": "string",
            "categorical": True
        },
        {
            "name": "DURATION",
            "type": "int"
        }
        
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "package-item",
    schema = json.dumps(itemmetadata_schema)
)

itemmetadataschema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws-cn:personalize:cn-north-1:657975005475:schema/package-item",
  "ResponseMetadata": {
    "RequestId": "f732d776-4bd8-4202-b015-53900405e2d1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 04:48:41 GMT",
      "x-amzn-requestid": "f732d776-4bd8-4202-b015-53900405e2d1",
      "content-length": "82",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [15]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "package-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = itemmetadataschema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws-cn:personalize:cn-north-1:657975005475:dataset/bestv-vod-item-summary/ITEMS",
  "ResponseMetadata": {
    "RequestId": "9ceddd89-83e1-45b8-8e1e-bc70678c96a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 04:48:55 GMT",
      "x-amzn-requestid": "9ceddd89-83e1-45b8-8e1e-bc70678c96a4",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [17]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "package-item-import1",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws-cn:personalize:cn-north-1:657975005475:dataset-import-job/package-item-import1",
  "ResponseMetadata": {
    "RequestId": "df5f6dbb-4cd3-49a5-9d35-e0aaa228504e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 04:49:31 GMT",
      "x-amzn-requestid": "df5f6dbb-4cd3-49a5-9d35-e0aaa228504e",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [18]:
%store items_dataset_arn
%store itemmetadataschema_arn

Stored 'items_dataset_arn' (str)
Stored 'itemmetadataschema_arn' (str)


In [10]:
rerank_recipe_arn = "arn:aws-cn:personalize:::recipe/aws-personalized-ranking"
rerank_create_solution_response = personalize.create_solution(
    name = "package-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))



In [ ]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

In [13]:
user_personalization_recipe_arn = "arn:aws-cn:personalize:::recipe/aws-user-personalization"
user_personalization_create_solution_response = personalize.create_solution(
    name = "package-userpersonalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_solution_arn = user_personalization_create_solution_response['solutionArn']
userpersonalization_create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)
userpersonalization_solution_version_arn = userpersonalization_create_solution_version_response['solutionVersionArn']
print(json.dumps(userpersonalization_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-userpersonalization/ddf2b6cd",
  "ResponseMetadata": {
    "RequestId": "c99a763d-ca73-4dea-bdfa-072f35a217f8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 06:40:14 GMT",
      "x-amzn-requestid": "c99a763d-ca73-4dea-bdfa-072f35a217f8",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [40]:
user_personalization_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = userpersonalization_solution_version_arn
)

print(json.dumps(user_personalization_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/personalize-poc-userpersonalization/c576c750",
  "metrics": {
    "coverage": 0.1915,
    "mean_reciprocal_rank_at_25": 0.0388,
    "normalized_discounted_cumulative_gain_at_10": 0.0513,
    "normalized_discounted_cumulative_gain_at_25": 0.067,
    "normalized_discounted_cumulative_gain_at_5": 0.0429,
    "precision_at_10": 0.0081,
    "precision_at_25": 0.0059,
    "precision_at_5": 0.0112
  },
  "ResponseMetadata": {
    "RequestId": "5160f97c-4f55-41af-914a-96ab30a774df",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Mar 2021 03:59:16 GMT",
      "x-amzn-requestid": "5160f97c-4f55-41af-914a-96ab30a774df",
      "content-length": "422",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [14]:
SIMS_recipe_arn = "arn:aws-cn:personalize:::recipe/aws-sims"
sims_create_solution_response = personalize.create_solution(
    name = "package-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))


{
  "solutionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-sims",
  "ResponseMetadata": {
    "RequestId": "8f00853e-9df8-4453-bc7f-aadb95381b69",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 06:40:41 GMT",
      "x-amzn-requestid": "8f00853e-9df8-4453-bc7f-aadb95381b69",
      "content-length": "86",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
{
  "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-sims/f43f7e36",
  "ResponseMetadata": {
    "RequestId": "b5eefa50-a584-4904-97f1-57c18f4862fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 18 Mar 2021 06:40:41 GMT",
      "x-amzn-requestid": "b5eefa50-a584-4904-97f1-57c18f4862fd",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [41]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/personalize-poc-sims/1bd4541f",
  "metrics": {
    "coverage": 0.0022,
    "mean_reciprocal_rank_at_25": 0.02,
    "normalized_discounted_cumulative_gain_at_10": 0.0304,
    "normalized_discounted_cumulative_gain_at_25": 0.0346,
    "normalized_discounted_cumulative_gain_at_5": 0.0246,
    "precision_at_10": 0.0051,
    "precision_at_25": 0.0027,
    "precision_at_5": 0.0068
  },
  "ResponseMetadata": {
    "RequestId": "f65acbb6-f9da-4455-90b8-f898fc05cd07",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Mar 2021 04:00:15 GMT",
      "x-amzn-requestid": "f65acbb6-f9da-4455-90b8-f898fc05cd07",
      "content-length": "406",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [25]:
!aws personalize list-solution-versions --region cn-north-1

{
    "solutionVersions": [
        {
            "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-rerank/2f9f5ec2",
            "status": "ACTIVE",
            "creationDateTime": 1616049565.135,
            "lastUpdatedDateTime": 1616052485.244
        },
        {
            "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-sims/f43f7e36",
            "status": "ACTIVE",
            "creationDateTime": 1616049642.134,
            "lastUpdatedDateTime": 1616051662.209
        },
        {
            "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-userpersonalization/ddf2b6cd",
            "status": "ACTIVE",
            "creationDateTime": 1616049615.097,
            "lastUpdatedDateTime": 1616052481.162
        },
        {
            "solutionVersionArn": "arn:aws-cn:personalize:cn-north-1:657975005475:solution/personalize-poc-rerank/2b69c7c5",
            "status": 

In [ ]:
%store userpersonalization_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn
%store user_personalization_solution_arn
%store sims_solution_arn
%store rerank_solution_arn

In [ ]:
in_progress_solution_versions = [
    userpersonalization_solution_version_arn,
    sims_solution_version_arn,
    rerank_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        else:
            print("Building {}".format(solution_version_arn))
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("The Current date is:" ,time.time())
        
    time.sleep(120)

Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-userpersonalization/ddf2b6cd
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-sims/f43f7e36
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-rerank/2f9f5ec2
The Current date is: 1616049672.1623232
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-userpersonalization/ddf2b6cd
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-sims/f43f7e36
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-rerank/2f9f5ec2
The Current date is: 1616049792.57503
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-userpersonalization/ddf2b6cd
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-sims/f43f7e36
Building arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-rerank/2f9f5ec2
The Current date is: 1616049912.8511617
Building arn:aws-cn:personalize:cn-north-1:65797